In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import voeventdb
import sqlalchemy

In [ ]:
import logging
logging.basicConfig()

In [ ]:
from voeventdb.database.models import Voevent, Base

In [ ]:
from voeventdb.database import db_utils
from voeventdb.tests.config import testdb_scratch_url, admin_db_url
if not db_utils.check_database_exists(testdb_scratch_url):
    db_utils.create_database(admin_db_url, testdb_scratch_url.database)
engine = sqlalchemy.engine.create_engine(testdb_scratch_url)
Base.metadata.create_all(engine)
engine.dispose()

In [ ]:
import sqlalchemy
from sqlalchemy.orm import sessionmaker
engine = sqlalchemy.engine.create_engine(testdb_scratch_url)
Session = sessionmaker(bind=engine)
s = Session()
s.query(Voevent).first()

In [ ]:
import sqlalchemy.orm as orm

In [ ]:
sm = orm.sessionmaker()
scoped_sm = orm.scoped_session(sm)
scoped_sm.configure(bind=engine) # configures the underlying `sm` sessionmaker object

A sessionmaker does not have a query property - we don't expect it to, after all it's for making sessions, not queries:

In [ ]:
# sm.query(Voevent).count() #<--Raises

So, make a session:

In [ ]:
regular_session = sm()
regular_session.query(Voevent).count()

Ok. We can do the same sort of thing with a scoped session:

In [ ]:
scoped_session = scoped_sm()
scoped_session.query(Voevent).count()

 However - shenanigans! - a `sqlalchemy.orm.scoped_session` (i.e. a scoped-session factory) has a .query attribute, created via the query_property method. AFAICT this is syntactic sugar, proxying to query attribute of the underlying session.
 
 This is documented here:  
http://docs.sqlalchemy.org/en/rel_1_0/orm/contextual.html?highlight=scoped_session#implicit-method-access
(*Though not very prominently, considering how heavily it's used in flask-related stuff. Breadcrumbs from e.g. flask-sqlalchemy docs might have been nice.*)

In [ ]:
scoped_sm.query(Voevent).count()